In [1]:
%matplotlib inline
%pylab inline 
import os
import seaborn
import numpy as np
import pandas as pd
import gradutil as gu
import simplejson as json
import interactiveBoreal as ib
import matplotlib.pyplot as plt
from ASF import ASF, NIMBUS
from time import time
from pyomo.opt import SolverFactory
from scipy.spatial.distance import euclidean

In [2]:
%%time
kehys = ib.ReferenceFrame()

In [3]:
with open('best_dict.json', 'r') as f:
    new_dict = json.load(f)

In [4]:
cn = new_dict['centers']
wn = new_dict['weights']
xt = new_dict['xtoc']
on = new_dict['out_centers']

In [5]:
kehys.centers = np.array(cn)
kehys.weights = np.array(wn)
kehys.xtoc = np.array(xt)
kehys.out_centers = np.array(on)

In [6]:
'''%%time
nclust = 600
seedn = 6
kehys.cluster(nclust=nclust, seedn=seedn)'''

In [7]:
import simplejson as json
with open('best_dict.json', 'w') as fi:
    json.dump(new_dict, fi)

In [8]:
data = kehys.centers
nvar = len(kehys.x_norm)
weights = kehys.weights/nvar
solver_name = 'cplex'
hectars = 68700

In [9]:
ide = kehys.normalize_ref(kehys.ideal)
nad = kehys.normalize_ref(kehys.nadir)

In [10]:
def scalarize(init_ref, frees=[]):
    ref = kehys.normalize_ref(init_ref)
    asf1   = ASF(ide, nad, ref, data, weights=weights, nvar=nvar, scalarization='asf', sense='maximize')
    stom1  = ASF(ide, nad, ref, data, weights=weights, nvar=nvar, scalarization='stom', sense='maximize')
    guess1 = ASF(ide, nad, ref, data, weights=weights, nvar=nvar, scalarization='guess',sense='maximize', frees=frees)

    asf_solver1 = ib.Solver(asf1.model, solver=solver_name)
    asf_res1 = asf_solver1.solve()
    print(asf_res1['Solver'][0]['Termination condition'])

    stom_solver1 = ib.Solver(stom1.model, solver=solver_name)
    stom_res1 = stom_solver1.solve()
    print(stom_res1['Solver'][0]['Termination condition'])

    guess_solver1 = ib.Solver(guess1.model, solver=solver_name)
    guess_res1 = guess_solver1.solve()
    print(guess_res1['Solver'][0]['Termination condition'])

    asf_values1   = kehys.values(model=asf1.model)
    stom_values1  = kehys.values(model=stom1.model)
    guess_values1 = kehys.values(model=guess1.model)
    
    return asf_values1, stom_values1, guess_values1

Lets try some reference points that were actually used in the interactive solving earlier

In [11]:
kyle1 = np.array((3266, 51.85, 2.103, 0.229))
ref1 = kyle1*hectars

In [12]:
kyle2 = np.array((3500, 50., 2., 0.22))
ref2 = kyle2*hectars

In [13]:
res1 = np.array(scalarize(ref1))/hectars
for r in res1:
    print(['{:7.2f}'.format(value) for value in r])

In [14]:
 print(['{:7.2f}'.format(value) for value in kyle1])

Well the results look good, even though the first objective was way too optimistic back then

In [15]:
res2 = np.array((scalarize(ref2)))/hectars
for r in res2:
     print(['{:7.2f}'.format(value) for value in r])

In [16]:
 print(['{:7.2f}'.format(value) for value in kyle2])

And here it looks pretty much the same than with the previous one. Great

## NIMBUS testing

Then we try the NIMBUS scalarization, using the results of the first ACH scalarization as a starting point.
The results were displayd per hectar, so we need to remember scale them to the full scale and then normalize for the NIMBUS

### Test 1

In [17]:
res1[0]

Lets set classification so that starting from the asf-result
of the previous problem:
1) the first objective should improve as much as possible,
2) the second detoriate to a 4.5+01, 
3) the third stay the same and 
4) the fourth change freely

In [18]:
start1 = res1[0]*hectars
kyle1_ref = np.array((kehys.ideal[0], 4.5e+01*hectars, start1[2], kehys.nadir[3]))
nimbus1_ref = kehys.normalize_ref(kyle1_ref)

In [19]:
''' The classes whose 'distance' to the Pareto front are to be
minized, i.e. the objectives to improve as much as possible and
the ones to improve to a limit'''
minmax1 = np.array([0], dtype=int)
''' The classes whose values are to be kept the same.'''
stay1 = np.array([2], dtype=int)
''' The classes whose values are to be deteriorated to a limit'''
detoriate1 = np.array([1], dtype=int)

In [20]:
nimbus1 = NIMBUS(ide, nad, nimbus1_ref, data, 
                 minmax1, stay1, detoriate1, kehys.normalize_ref(start1), 
                 weights=weights, nvar=nvar)
nimbus1_solver = ib.Solver(nimbus1.model, solver=solver_name)
nimbus1_res = nimbus1_solver.solve()  #output=True)  #, keepfiles=True)
nimbus1_values = kehys.values(model=nimbus1.model)
str(nimbus1_res['Solver'][0]['Termination condition'])

In [21]:
 print(['{:7.2f}'.format(value) for value in nimbus1_values/hectars])

The nimbus was supposed to improve the first objective as much as possible, detoriate the
second to a 4.5e+01, the third stay the same and the fourth change freely. The starting point is presented in the below, so everything looks pretty good!

In [22]:
 print(['{:7.2f}'.format(value) for value in start1/hectars])

### Test 2


Lets set classification so that starting from the same asf-result than before:
1) the first objective should change freely, 
2) the second improve to 5.5e+01, 
3) the third improve as much as possible and 
4) the fourth stay the same

In [23]:
kyle2_ref = np.array((kehys.nadir[0], 5.5e+01*hectars, kehys.ideal[2], start1[3]))
nimbus2_ref = kehys.normalize_ref(kyle2_ref)

In [24]:
''' The classes whose 'distance' to the Pareto front are to be
minized, i.e. the objectives to improve as much as possible and
the ones to improve to a limit'''
minmax2 = np.array([1,2], dtype=int)
''' The classes whose values are to be kept the same.'''
stay2 = np.array([3], dtype=int)
''' The classes whose values are to be deteriorated to a limit'''
detoriate2 = np.array([], dtype=int)

In [25]:
nimbus2 = NIMBUS(ide, nad, nimbus2_ref, data, minmax2, 
                 stay2, detoriate2, kehys.normalize_ref(start1), weights=weights, nvar=nvar)
nimbus2_solver = ib.Solver(nimbus2.model, solver=solver_name)
nimbus2_res = nimbus2_solver.solve()  # output=True, keepfiles=True)
nimbus2_values = kehys.values(model=nimbus2.model)

In [26]:
 print(['{:7.2f}'.format(value) for value in nimbus2_values/hectars])

As there seems to be something wrong with the second objective getting too good and the fourth not staying the same, we try using the other scalarizing for the same preferences and see if we get better results

In [27]:
 print(['{:7.2f}'.format(value) for value in start1/hectars])

## Verifying by using scalarizations

Next lets try what we get if using the classification information of the NIMBUS as a reference point for the other scalarizings. Notice how (guess) scalarization has to be told what objective is free to change as it wants.

In [28]:
res3 = np.array((scalarize(kyle2_ref, frees=[0])))/hectars

In [29]:
for r in res3:
    print(['{:7.2f}'.format(value) for value in r])

Now it looks like that all the scalarizings agree about the results so probably this is so then. So be it.